In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from Models.pgmpy_BN import Pgmpy_BN
from Evaluation.utils import parse_query

ModuleNotFoundError: No module named 'Models'

In [2]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
from DataPrepare.join_data_preparation import JoinDataPreparator
schema = gen_job_light_imdb_schema('/home/ziniu.wzn/imdb-benchmark')
from DataPrepare.join_data_preparation import JoinDataPreparator
hdf_path = "/home/ziniu.wzn/imdb-benchmark/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [3]:
from Models.pgmpy_BN import Pgmpy_BN, build_meta_info
def train_all(algo="chow-liu", max_parents=1):
    for i, relationship_obj in enumerate(schema.relationships):
        sample_size = 10000000
        relation = relationship_obj.identifier
        df, meta_types, null_values, full_join_est = prep.generate_n_samples(
                        sample_size, relationship_list=[relation], post_sampling_factor=10)
        print(full_join_est)
        print(len(df), len(df.columns))
        print(meta_types)
        meta_info = build_meta_info(df.columns)
        bn = Pgmpy_BN(relation, meta_info, full_join_est)
        model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}.pkl"
        print(model_path)
        bn.build_from_data(df, algorithm=algo, max_parents=max_parents, ignore_cols=['id'], sample_size=200000)
        pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        try:
            BN.model = Bn.model.to_junction_tree()
            model_path = f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_junction.pkl"
            pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
        except:
            print("This BN is not able to transform into junction tree, probably because it's not connected, just use BN")

In [ ]:
train_all("exact", 3)

In [4]:
from Evaluation.parse_query_imdb import prepare_join_queries

In [5]:
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
ensemble_location = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/ensemble_relationships_imdb-light_10000000.pkl"
#query_filename = "/home/ziniu.wzn/imdb-benchmark/cardinality/job_all.sql"
query_filename = "/home/ziniu.wzn/BayesCard/Benchmark/IMDB/job-light.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=False, true_card_exist=True)

In [6]:
parsed_queries[30]

[4050205.0,
 {'bn_index': 4,
  'inverse': False,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_companies.movie_companies_nn': 1,
   'movie_companies.company_type_id': 2.0},
  'expectation': ['title.mul_movie_keyword.movie_id']},
 {'bn_index': 3,
  'inverse': False,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_keyword.movie_keyword_nn': 1,
   'movie_keyword.keyword_id': 398.0},
  'expectation': []},
 {'bn_index': 3,
  'inverse': True,
  'query': {'title.production_year': (1950.1, 1999.9),
   'movie_keyword.movie_keyword_nn': 1},
  'expectation': []}]

In [10]:
bn_ensemble = dict()
algo = "chow-liu"
for i in range(5):
    with open(f"/home/ziniu.wzn/imdb-benchmark/BN_model/{i}_{algo}_1.pkl", "rb") as f:
        bn = pickle.load(f)
    bn_ensemble[i] = bn